In [ ]:
# Install Packages
!pip install psycopg2
!pip install pandas_profiling
!pip install kaggle

In [ ]:
# Import Packages
import numpy as np
import pandas as pd
import psycopg2
import kaggle
import zipfile

from pandas_profiling import ProfileReport
from pandas_profiling.utils.cache import cache_file

In [ ]:
# connect to Kaggle API
from kaggle.api.kaggle_api_extended import KaggleApi
api = KaggleApi()
api.authenticate()

In [ ]:
# Load Datasets into current dir.

api.dataset_download_file('ashirwadsangwan/imdb-dataset',file_name='title.akas.tsv.gz',path='./')
api.dataset_download_file('ashirwadsangwan/imdb-dataset',file_name='title.basics.tsv.gz',path='./')
api.dataset_download_file('ashirwadsangwan/imdb-dataset',file_name='title.principals.tsv.gz',path='./')
api.dataset_download_file('ashirwadsangwan/imdb-dataset',file_name='name.basics.tsv.gz',path='./')
api.dataset_download_file('ashirwadsangwan/imdb-dataset',file_name='title.ratings.tsv.gz',path='./')

In [ ]:
# Unzip Files

with zipfile.ZipFile('./title.ratings.tsv.gz.zip', 'r') as zipref:
     zipref.extractall('./')
with zipfile.ZipFile('./title.basics.tsv.gz.zip', 'r') as zipref:
    zipref.extractall('./') 
with zipfile.ZipFile('./title.akas.tsv.gz.zip', 'r') as zipref:
    zipref.extractall('./')
with zipfile.ZipFile('./title.principals.tsv.gz.zip', 'r') as zipref:
    zipref.extractall('./')
with zipfile.ZipFile('./name.basics.tsv.gz.zip', 'r') as zipref:
    zipref.extractall('./')

In [ ]:
# Create DataTable from Data title.ratings
df_Ratings = pd.read_csv('./title.ratings.tsv.gz', sep='\t', na_values='\\N', header=0, index_col=False)
df_Ratings['tconst'].head()

In [ ]:
# Analyze title.ratings Table
ratingProfile = ProfileReport(df_Ratings, title="IMDB Ratings", explorative=True)
ratingProfile.to_file("./IMDB Ratings.html")

In [ ]:
# Write local title.ratings Table to title.ratings table in psql
try:
    connection = psycopg2.connect(user="postgres",
                                  password="***************",
                                  host="localhost",
                                  port="5432",
                                  database="ResSemHW2")
    
    cursor = connection.cursor()
    
    # Creating a list of tupples from the dataframe values
    tpls = [tuple(x) for x in df_Ratings.to_numpy()]
    
    # dataframe columns with Comma-separated
    cols = ','.join(list(df_Ratings.columns))
    
    sql_insert_query = "INSERT INTO public.titleRatings VALUES (%s,%s,%s)" % (cols)

    # executemany() to insert multiple rows
    result = cursor.executemany(sql_insert_query,tpls)
    connection.commit()
    print(cursor.rowcount, "Record inserted successfully into table")

except (Exception, psycopg2.Error) as error:
    print("Failed inserting record into table {}".format(error))

finally:
    # closing database connection.
    if connection:
        cursor.close()
        connection.close()
        print("PostgreSQL connection is closed")

In [ ]:
# Create DataTable from Data title.Basics
col_list = ["tconst","titleType","primaryTitle","originalTitle","isAdult","startYear","genres" ]
df_TitleBasics = pd.read_csv('./title.basics.tsv.gz', na_values='\\N', sep='\t', header=0, 
                             dtype ={"tconst":str,"titleType":str,"primaryTitle":str,"originalTitle":str,
                                     "isAdult":str,"startYear":str,"genres":str}, usecols = col_list)
df_TitleBasics.head()

In [ ]:
# Analyze title.Basics Table
titleBasicProfile = ProfileReport(df_TitleBasics, title="IMDB Title Basics", explorative=True)
titleBasicProfile.to_file("./IMDB Title Basics.html")

In [ ]:
# Create DataTable from Data title.akas
col_list = ["titleId","ordering","title","region","language","types","attributes","isOriginalTitle"]
df_TitleAkas = pd.read_csv('./title.akas.tsv.gz', na_values='\\N', sep='\t', header=0,
                           dtype ={"titleId":str,"ordering":str,"title":str,"region":str,"language":str,
                                   "types":str,"attributes":str,"isOriginalTitle":str}, usecols=col_list)
df_TitleAkas.head()

In [ ]:
# Prepare for profiling
df_TitleAkas = df_TitleAkas.sample(frac=0.25)
df_TitleAkas = df_TitleAkas.fillna('null')

In [ ]:
# Analyze title.akas Table
titleAkasProfile = ProfileReport(df_TitleAkas.sample, title="IMDB Title Akas", explorative=True)
titleAkasProfile.to_file("./IMDB Title Akas.html")

In [ ]:
# Create DataTable from Data title.principals

col_list = ["tconst","ordering","nconst","category"]

df_TitlePrincipals = pd.read_csv('./title.principals.tsv.gz', na_values='\\N', sep='\t', header=0,
                                 dtype ={"tconst":str,"ordering":str,"nconst":str,"category":str},
                                 usecols=col_list)
df_TitlePrincipals.head()

In [ ]:
# Prepare Data for Profiling
df_TitlePrincipals = df_TitlePrincipals.sample(n = 999999)
df_TitlePrincipals = df_TitlePrincipals.fillna('')
df_TitlePrincipals = df_TitlePrincipals.replace('\["',"",regex=True)
df_TitlePrincipals = df_TitlePrincipals.replace('\"]',"",regex=True)
df_TitlePrincipals = df_TitlePrincipals.replace('"',"",regex=True)
df_TitlePrincipals

In [ ]:
# Analyze title.principals Table
titlePrincipalProfile = ProfileReport(df_TitlePrincipals, title="IMDB Title Principals", explorative=True)
titlePrincipalProfile.to_file("./IMDB Title Principals.html")

In [ ]:
# Create DataTable from Data name.basics

col_list = ["nconst","primaryName","birthYear","deathYear","primaryProfession","knownForTitles"]

df_NameBasics = pd.read_csv('./name.basics.tsv.gz', na_values='\\N', sep='\t', header=0,
                            dtype ={"nconst":str,"primaryName":str,"birthYear":str,"deathYear":str,
                                    "primaryProfession":str,"knownForTitles":str}, usecols=col_list)
df_NameBasics.head()

In [ ]:
# Preapare Data for Profiling
df_Analyze = df_NameBasics.sample(n = 999999)
df_Analyze = df_Analyze.fillna('null')

In [ ]:
# Analyze name.basics Table
nameBasicsProfile = ProfileReport(df_Analyze, title="IMDB Name Basics",explorative=True)
nameBasicsProfile.to_file("./IMDB Name Basics.html")

In [ ]:
# Write local title.basic Table to title.basics table in psql
try:
    connection = psycopg2.connect(user="postgres",
                                  password="***************",
                                  host="localhost",
                                  port="5432",
                                  database="ResSemHW2")
    
    cursor = connection.cursor()
    
    # Creating a list of tupples from the dataframe values
    tpls = [tuple(x) for x in df_TitleBasics.to_numpy()]
    
    # dataframe columns with Comma-separated
    cols = ','.join(list(df_TitleBasics.columns))
    print(cols)
    sql_insert_query = """INSERT INTO public.titleBasics ("tconst","titleType","primaryTitle",
    "originalTitle","isAdult","startYear","genres") VALUES (%s,%s,%s,%s,%s,%s,%s)"""

    # executemany() to insert multiple rows
    print('5')
    result = cursor.executemany(sql_insert_query,tpls)
    connection.commit()
    print(cursor.rowcount, "Record inserted successfully into table")

except (Exception, psycopg2.Error) as error:
    print("Failed inserting record into table {}".format(error))

finally:
    # closing database connection.
    if connection:
        cursor.close()
        connection.close()
        print("PostgreSQL connection is closed")
    

In [ ]:
# Write local title.akas Table to title.akas table in psql
try:
    connection = psycopg2.connect(user="postgres",
                                  password="***************",
                                  host="localhost",
                                  port="5432",
                                  database="ResSemHW2")
    
    cursor = connection.cursor()
    
    # Creating a list of tupples from the dataframe values
    tpls = [tuple(x) for x in df_TitleAkas.to_numpy()]
    
    # dataframe columns with Comma-separated
    cols = ','.join(list(df_TitleAkas.columns))
    print(cols)
    sql_insert_query1 = """INSERT INTO public."titleAkas" ("titleId","ordering","title","region","language","types","attributes","isOriginalTitle") VALUES (%s,%s,%s,%s,%s,%s,%s,%s)""" #% (cols)

    # executemany() to insert multiple rows
    print('5')
    result = cursor.executemany(sql_insert_query1,tpls)
    connection.commit()
    print(cursor.rowcount, "Record inserted successfully into table")

except (Exception, psycopg2.Error) as error:
    print("Failed inserting record into table {}".format(error))

finally:
    # closing database connection.
    if connection:
        cursor.close()
        connection.close()
        print("PostgreSQL connection is closed")
    

In [ ]:
# Write local title.principals Table to title.principals table in psql
try:
    connection = psycopg2.connect(user="postgres",
                                  password="***************",
                                  host="localhost",
                                  port="5432",
                                  database="ResSemHW2")
    
    cursor = connection.cursor()
    
    # Creating a list of tupples from the dataframe values
    tpls = [tuple(x) for x in df_TitlePrincipals.to_numpy()]
    
    # dataframe columns with Comma-separated
    cols = ','.join(list(df_TitlePrincipals.columns))
    sql_insert_query = """INSERT INTO public."titlePrincipals" ("tconst","ordering","nconst","category","job","characters") VALUES (%s,%s,%s,%s,%s,%s)""" #% (cols)

    # executemany() to insert multiple rows
    print('5')
    result = cursor.executemany(sql_insert_query,tpls)
    connection.commit()
    print(cursor.rowcount, "Record inserted successfully into table")

except (Exception, psycopg2.Error) as error:
    print("Failed inserting record into table {}".format(error))

finally:
    # closing database connection.
    if connection:
        cursor.close()
        connection.close()
        print("PostgreSQL connection is closed")
    

In [ ]:
# Write local name.basics Table to name.basics table in psql
try:
    connection = psycopg2.connect(user="postgres",
                                  password="***************",
                                  host="localhost",
                                  port="5432",
                                  database="ResSemHW2")
    
    cursor = connection.cursor()
    
    # Creating a list of tupples from the dataframe values
    tpls = [tuple(x) for x in df_NameBasics.to_numpy()]
    
    # dataframe columns with Comma-separated
    cols = ','.join(list(df_NameBasics.columns))
    print(cols)
    sql_insert_query = """INSERT INTO public.nameBasics ("nconst","primaryName","birthYear","deathYear","primaryProfession","knownForTitles") VALUES (%s,%s,%s,%s,%s,%s)""" #% (cols)

    # executemany() to insert multiple rows
    print('5')
    result = cursor.executemany(sql_insert_query,tpls)
    connection.commit()
    print(cursor.rowcount, "Record inserted successfully into table")

except (Exception, psycopg2.Error) as error:
    print("Failed inserting record into table {}".format(error))

finally:
    # closing database connection.
    if connection:
        cursor.close()
        connection.close()
        print("PostgreSQL connection is closed")
    